In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, classification_report,
                             confusion_matrix, roc_curve)
import pickle
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
warnings.filterwarnings('ignore')

# --- 1. Data Loading and Initial Cleaning ---
print("="*80)
print("SUPPLY CHAIN LATE DELIVERY RISK PREDICTION MODEL")
print("="*80)
print("\n1. Loading and Cleaning Data...")
data = pd.read_csv("DataCoSupplyChainDataset.csv", encoding="ISO-8859-1")
data.columns = data.columns.str.strip()

# Basic data exploration
print(f"   Dataset shape: {data.shape}")
print(f"   Columns: {data.shape[1]}")
missing = data.isnull().sum()
if missing.sum() > 0:
    print(f"\n   Missing values:")
    print(missing[missing > 0])
else:
    print("   No missing values detected")

# --- 2. Enhanced Feature Engineering ---
print("\n2. Engineering Features...")
data['order_date'] = pd.to_datetime(data['order date (DateOrders)'], errors='coerce')

# Temporal features
data['order_hour'] = data['order_date'].dt.hour
data['order_day_of_week'] = data['order_date'].dt.dayofweek
data['order_month'] = data['order_date'].dt.month
data['order_quarter'] = data['order_date'].dt.quarter
data['order_day_of_month'] = data['order_date'].dt.day
data['is_weekend'] = (data['order_day_of_week'] >= 5).astype(int)
data['is_business_hours'] = ((data['order_hour'] >= 9) & (data['order_hour'] <= 17)).astype(int)
data['is_month_end'] = (data['order_day_of_month'] >= 25).astype(int)

# Product/Order interaction features
if 'Order Item Quantity' in data.columns and 'Order Item Discount' in data.columns:
    data['discount_per_item'] = data['Order Item Discount'] / (data['Order Item Quantity'] + 1)
    data['high_discount_flag'] = (data['Order Item Discount'] > data['Order Item Discount'].quantile(0.75)).astype(int)

if 'Sales' in data.columns and 'Order Item Quantity' in data.columns:
    data['price_per_item'] = data['Sales'] / (data['Order Item Quantity'] + 1)

# Shipping complexity score
if 'Days for shipment (scheduled)' in data.columns:
    data['urgent_shipment'] = (data['Days for shipment (scheduled)'] <= 2).astype(int)

# Drop invalid rows
initial_rows = len(data)
data = data.dropna(subset=['order_date', 'Late_delivery_risk'])
print(f"   Rows removed due to missing critical data: {initial_rows - len(data)}")
print(f"   Rows after cleaning: {len(data)}")

# --- 3. Target and Feature Selection ---
print("\n3. Selecting Features and Target...")
y = data['Late_delivery_risk'].astype(int)
print(f"\n   Target distribution:")
print(y.value_counts())
print(f"\n   Target proportions:")
print(y.value_counts(normalize=True))

# Feature selection (EXCLUDING potential leakage features)
# Features known AT ORDER TIME only
features = [
    # Temporal features
    'Days for shipment (scheduled)',
    'order_hour',
    'order_day_of_week',
    'order_month',
    'order_quarter',
    'order_day_of_month',
    'is_weekend',
    'is_business_hours',
    'is_month_end',

    # Operational features
    'Shipping Mode',
    'Type',  # Payment Type

    # Product/Department features
    'Department Id',
    'Category Id',
    'Product Category Id',

    # Order characteristics
    'Order Item Quantity',
    'Order Item Discount',
    'Order Item Profit Ratio',
    'Sales',
    'Order Item Total',

    # Engineered features
    'discount_per_item',
    'high_discount_flag',
    'price_per_item',
    'urgent_shipment'
]

# Verify features exist
features = [f for f in features if f in data.columns]
print(f"   Total features selected: {len(features)}")
print(f"   Features: {features}")

X = data[features].copy()

# Handle missing values more carefully
numeric_features = X.select_dtypes(include=[np.number]).columns
categorical_features = X.select_dtypes(include=['object']).columns

print(f"\n   Numeric features: {len(numeric_features)}")
print(f"   Categorical features: {len(categorical_features)}")

# Fill missing values
for col in numeric_features:
    if X[col].isnull().sum() > 0:
        X[col].fillna(X[col].median(), inplace=True)

for col in categorical_features:
    if X[col].isnull().sum() > 0:
        X[col].fillna('Unknown', inplace=True)

# --- 4. Train-Test Split ---
print("\n4. Splitting Data...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"   Train set: {len(X_train)} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"   Test set:  {len(X_test)} samples ({len(X_test)/len(X)*100:.1f}%)")
print(f"\n   Train target distribution:")
print(y_train.value_counts(normalize=True))

# --- 5. Preprocessing Setup ---
print("\n5. Setting up Preprocessing Pipeline...")
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, max_categories=50), categorical_cols)
], remainder='passthrough')

# --- 6. Model Training ---
print("\n6. Training Models...")

# Calculate class imbalance
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"   Class imbalance ratio: {scale_pos_weight:.2f}")

# XGBoost with optimized parameters
pipe_xgb = Pipeline([
    ('pre', preprocessor),
    ('clf', XGBClassifier(
        eval_metric='logloss',
        random_state=42,
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        scale_pos_weight=scale_pos_weight,
        colsample_bytree=0.8,
        subsample=0.8,
        min_child_weight=3,
        gamma=0.1,
        reg_alpha=0.1,  # L1 regularization
        reg_lambda=1.0   # L2 regularization
    ))
])

# RandomForest with balanced weights
pipe_rf = Pipeline([
    ('pre', preprocessor),
    ('clf', RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=5,
        max_features='sqrt',
        random_state=42,
        class_weight='balanced',
        n_jobs=-1
    ))
])

# Train models
print("   Training XGBoost...")
pipe_xgb.fit(X_train, y_train)
print("   ✓ XGBoost trained")

print("   Training RandomForest...")
pipe_rf.fit(X_train, y_train)
print("   ✓ RandomForest trained")

# --- 7. Cross-Validation ---
print("\n7. Cross-Validation (5-Fold)...")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("   XGBoost CV Scores:")
cv_scores_xgb = cross_val_score(pipe_xgb, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
print(f"      Mean AUC: {cv_scores_xgb.mean():.4f} (+/- {cv_scores_xgb.std():.4f})")

print("   RandomForest CV Scores:")
cv_scores_rf = cross_val_score(pipe_rf, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
print(f"      Mean AUC: {cv_scores_rf.mean():.4f} (+/- {cv_scores_rf.std():.4f})")

# --- 8. Enhanced Evaluation ---
print("\n" + "="*80)
print("8. MODEL EVALUATION")
print("="*80)

def evaluate(name, model, X_train, y_train, X_test, y_test):
    # Train predictions
    train_preds = model.predict(X_train)
    train_acc = accuracy_score(y_train, train_preds)

    # Test predictions
    test_preds = model.predict(X_test)
    test_probs = model.predict_proba(X_test)[:, 1]

    # Metrics
    acc = accuracy_score(y_test, test_preds)
    prec = precision_score(y_test, test_preds, zero_division=0)
    rec = recall_score(y_test, test_preds, zero_division=0)
    f1 = f1_score(y_test, test_preds, zero_division=0)
    auc = roc_auc_score(y_test, test_probs)

    # Confusion matrix
    cm = confusion_matrix(y_test, test_preds)

    print(f"\n{'='*80}")
    print(f"  {name}")
    print(f"{'='*80}")
    print(f"  Train Accuracy: {train_acc:.4f}")
    print(f"  Test Accuracy:  {acc:.4f}")
    print(f"  Precision:      {prec:.4f} (of predicted late, % actually late)")
    print(f"  Recall:         {rec:.4f} (of actual late, % predicted late)")
    print(f"  F1 Score:       {f1:.4f}")
    print(f"  ROC-AUC:        {auc:.4f}")

    print(f"\n  Confusion Matrix:")
    print(f"                Predicted")
    print(f"                On-Time  Late")
    print(f"  Actual On-Time  {cm[0][0]:6d}  {cm[0][1]:5d}")
    print(f"  Actual Late     {cm[1][0]:6d}  {cm[1][1]:5d}")

    print(f"\n  Classification Report:")
    print(classification_report(y_test, test_preds, target_names=['On-Time', 'Late']))

    # Calculate business metrics
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    print(f"  Specificity (True Negative Rate): {specificity:.4f}")

    return auc, test_probs, test_preds

auc_xgb, probs_xgb, preds_xgb = evaluate("XGBoost", pipe_xgb, X_train, y_train, X_test, y_test)
auc_rf, probs_rf, preds_rf = evaluate("RandomForest", pipe_rf, X_train, y_train, X_test, y_test)

# --- 9. Feature Importance ---
print("\n" + "="*80)
print("9. FEATURE IMPORTANCE")
print("="*80)

def plot_feature_importance(model, name, top_n=20):
    if hasattr(model.named_steps['clf'], 'feature_importances_'):
        # Get feature names
        cat_features = model.named_steps['pre'].named_transformers_['cat'].get_feature_names_out(categorical_cols)
        feature_names = numeric_cols + list(cat_features)

        # Get importances
        importances = model.named_steps['clf'].feature_importances_

        # Create DataFrame
        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importances
        }).sort_values(by='Importance', ascending=False)

        print(f"\n{name} - Top {top_n} Features:")
        print(importance_df.head(top_n).to_string(index=False))

        # Plot
        plt.figure(figsize=(10, 8))
        top_features = importance_df.head(top_n)
        plt.barh(range(len(top_features)), top_features['Importance'])
        plt.yticks(range(len(top_features)), top_features['Feature'])
        plt.xlabel('Importance')
        plt.title(f'{name} - Top {top_n} Feature Importances')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.savefig(f'models_fixed/{name.lower()}_feature_importance.png', dpi=300, bbox_inches='tight')
        print(f"   ✓ Feature importance plot saved: {name.lower()}_feature_importance.png")
        plt.close()

        return importance_df

os.makedirs("models_fixed", exist_ok=True)
importance_xgb = plot_feature_importance(pipe_xgb, "XGBoost", top_n=20)
importance_rf = plot_feature_importance(pipe_rf, "RandomForest", top_n=20)

# --- 10. ROC Curve Visualization ---
print("\n10. Generating ROC Curves...")
plt.figure(figsize=(10, 8))

# XGBoost ROC
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, probs_xgb)
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC = {auc_xgb:.4f})', linewidth=2)

# RandomForest ROC
fpr_rf, tpr_rf, _ = roc_curve(y_test, probs_rf)
plt.plot(fpr_rf, tpr_rf, label=f'RandomForest (AUC = {auc_rf:.4f})', linewidth=2)

# Random baseline
plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC = 0.5000)', linewidth=1)

plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - Late Delivery Prediction', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('models_fixed/roc_curves.png', dpi=300, bbox_inches='tight')
print("   ✓ ROC curve saved: roc_curves.png")
plt.close()

# --- 11. Save Best Model ---
print("\n" + "="*80)
print("11. SAVING MODELS")
print("="*80)

best_model_name = "XGBoost" if auc_xgb >= auc_rf else "RandomForest"
best_model = pipe_xgb if auc_xgb >= auc_rf else pipe_rf
best_auc = max(auc_xgb, auc_rf)

# Save models
pickle.dump(best_model, open("models_fixed/best_model.pkl", "wb"))
pickle.dump(pipe_xgb, open("models_fixed/xgb_model.pkl", "wb"))
pickle.dump(pipe_rf, open("models_fixed/rf_model.pkl", "wb"))

print(f"   ✓ Best model: {best_model_name} (AUC: {best_auc:.4f})")
print(f"   ✓ Models saved to 'models_fixed/' directory")

# --- 12. Save Metadata ---
print("\n12. Saving Model Metadata...")

# Feature list
with open("models_fixed/feature_list.txt", "w") as f:
    f.write("# Features used for prediction (known at order time)\n")
    f.write("# Generated: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n\n")
    f.write("\n".join(features))

# Model metadata
metadata = {
    'training_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'best_model': best_model_name,
    'best_auc': float(best_auc),
    'xgb_auc': float(auc_xgb),
    'rf_auc': float(auc_rf),
    'xgb_cv_mean': float(cv_scores_xgb.mean()),
    'rf_cv_mean': float(cv_scores_rf.mean()),
    'train_samples': len(X_train),
    'test_samples': len(X_test),
    'num_features': len(features),
    'class_imbalance_ratio': float(scale_pos_weight),
    'feature_list': features,
    'numeric_features': numeric_cols,
    'categorical_features': categorical_cols
}

import json
with open("models_fixed/model_metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)

print("   ✓ Feature list saved: feature_list.txt")
print("   ✓ Model metadata saved: model_metadata.json")

# --- 13. Model Performance Summary ---
print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)
print(f"\nBest Model: {best_model_name}")
print(f"Test ROC-AUC: {best_auc:.4f}")
print(f"\nAll outputs saved to: models_fixed/")
print("\nFiles generated:")
print("  - best_model.pkl (production model)")
print("  - xgb_model.pkl")
print("  - rf_model.pkl")
print("  - feature_list.txt")
print("  - model_metadata.json")
print("  - xgboost_feature_importance.png")
print("  - randomforest_feature_importance.png")
print("  - roc_curves.png")
print("\n" + "="*80)
print("MODEL TRAINING COMPLETE!")
print("="*80)

SUPPLY CHAIN LATE DELIVERY RISK PREDICTION MODEL

1. Loading and Cleaning Data...
   Dataset shape: (180519, 53)
   Columns: 53

   Missing values:
Customer Lname              8
Customer Zipcode            3
Order Zipcode          155679
Product Description    180519
dtype: int64

2. Engineering Features...
   Rows removed due to missing critical data: 0
   Rows after cleaning: 180519

3. Selecting Features and Target...

   Target distribution:
Late_delivery_risk
1    98977
0    81542
Name: count, dtype: int64

   Target proportions:
Late_delivery_risk
1    0.548291
0    0.451709
Name: proportion, dtype: float64
   Total features selected: 23
   Features: ['Days for shipment (scheduled)', 'order_hour', 'order_day_of_week', 'order_month', 'order_quarter', 'order_day_of_month', 'is_weekend', 'is_business_hours', 'is_month_end', 'Shipping Mode', 'Type', 'Department Id', 'Category Id', 'Product Category Id', 'Order Item Quantity', 'Order Item Discount', 'Order Item Profit Ratio', 'Sales',